In [6]:
from tqdm.notebook import tqdm


In [13]:
from transformers import AutoTokenizer, pipeline
model_name = "PleIAs/Pleias-RAG-1B"
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [8]:
import os
import sys
from pathlib import Path
env_path = Path("../RAG_illama/.env")
from dotenv import load_dotenv
load_dotenv(dotenv_path=env_path)
PINEAPI = os.getenv("PINECONE")


In [17]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [20]:
# pipe = pipeline('text-generation', model=model_name, tokenizer=model_name, device=0)
# output = pipe(
#     max_length=1000, 
#     num_return_sequences=1, 
#     top_k=50, 
#     top_p=0.95, 
#     temperature=0.5, 
#     do_sample=True, 
#     return_full_text=False
# )

In [21]:
llm_model = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.4,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=500,
)

Device set to use cpu


In [22]:
prompt = """
<|system|>
You are a helpful assistant that answers on medical questions based on the real information provided from different sources and in the context.
Give the rational and well written response. If you don't have proper info in the context, answer "I don't know"
Respond only to the question asked.

<|user|>
Context:
{context}
---
Here is the question you need to answer.

Question: {question}
<|assistant|>
"""

In [23]:
from langchain_community.embeddings import HuggingFaceEmbeddings
model_name = "thenlper/gte-small"

In [24]:
embedding_model = HuggingFaceEmbeddings(
    model_name=model_name,
    multi_process=True,
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True},  # Set `True` for cosine similarity
)

C:\Users\khatt\AppData\Local\Temp\ipykernel_20496\3142973650.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
c:\Users\khatt\Documents\RAG_FineTuning\venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\khatt\.cache\huggingface\hub\models--thenlper--gte-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment 

In [27]:
PINEAPI = os.getenv("PINECONE")
from pinecone import Pinecone
pc = Pinecone(api_key=PINEAPI)
index = pc.Index("vectorized")
user_input = input("Ask a question")
vectorized_input = embedding_model.embed_query(user_input)

context = index.query(
    namespace="ns1",
    vector=vectorized_input,
    top_k=1,
    include_metadata=True
)

In [31]:
context['matches'][0]['metadata']['text']

'5\tRenal abscess in children. Three cases of renal abscesses in children are described to illustrate the variable presenting features. An additional 23 pediatric cases, reported over the past ten years, were reviewed for clinical features and therapy. Fever, loin pain, and leukocytosis were common presenting features, but less than half of all abscesses were associated with either an abnormal urinalysis or a positive urine culture. The presenting features were sometimes confused with appendicitis, peritonitis, or a Wilms tumor. An organism was identified in 17 cases--Escherichia coli in 9 children and Staphylococcus aureus in 8 children. The majority of E. coli infections occurred in girls and the majority of S. aureus infections occurred in boys. Reflux was documented in 5 patients, and 2 children had a possible extrarenal source of infection. Antibiotics alone produced a cure in 10 children (38%), but 16 children (62%) required a surgical procedure.'

In [32]:
answer = llm_model(prompt.format(context = context['matches'][0]['metadata']['text'], question = user_input))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [38]:
answer[0]['generated_text'].split('Trivial')[0]

"The query asks about the typical presentation of renal abscesses in children, specifically focusing on how these presentations vary across different cases. While this might seem straightforward, it requires careful analysis of the available sources to identify patterns and variations.\n\nMost sources can be immediately discarded as they discuss unrelated topics (like kidney stones, abscesses in adults, or other medical conditions). However, we do have two relevant sources:\n\n- Source 1 describes various types of renal abscesses in children, including their presentation characteristics.\n- Source 6 provides additional context about renal abscesses in children, noting their rarity compared to adult cases.\n\nBy cross-referencing these sources, we can see that while there are some differences between types of renal abscesses (as shown by source 1), there appears to be consistency in terms of presenting features. This suggests that the variability in presentation isn't due to different t